<div style="text-align: left">
    <img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/logo.png?raw=true' width=800/>  
</div>

Author: Itay Segev

E-mail: [itaysegev@campus.technion.ac.il](mailto:itaysegev@campus.technion.ac.il)



# From Tabular Q-Learning to DQN




<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/presentations/dqn-tutorial/images/dqn_nature.png?raw=true' width=900/>


<a id="section:intro"></a>

# <img src="https://img.icons8.com/?size=50&id=55412&format=png&color=000000" style="height:50px;display:inline"> Introduction
---

In this tutorial, you'll learn how to implement the **Fitted Q-Iteration** (FQI) algorithm, which is the predecessor of the [Deep Q-Network (DQN)](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) algorithm. To address the limitations of FQI in dealing with large and complex state spaces, we transition to **DQN**, which enhances FQI by using neural networks and introducing several key components.

You will then learn how to implement step-by-step the FQI algorithm. In this notebook, you will implement the main components of the Deep Q-Network (DQN) algorithm: a replay buffer, epsilon-greedy exploration strategy, and a Q-network. These components are crucial for stabilizing the learning process and improving the efficiency of the algorithm.




# <img src="https://img.icons8.com/?size=50&id=43171&format=png&color=000000" style="height:30px;display:inline"> Setup


You will need to make a copy of this notebook in your Google Drive before you can edit the notebook. You can do so with **File &rarr; Save a copy in Drive**.

In [ ]:
#@title mount your Google Drive
import os
connect_drive = False #@param {type: "boolean"}
if connect_drive:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

  # set up mount symlink
  DRIVE_PATH = '/content/gdrive/My\ Drive/cs236203_s24'
  DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
  if not os.path.exists(DRIVE_PYTHON_PATH):
    %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/cs236203_s24'
if not os.path.exists(SYM_PATH) and connect_drive:
  !ln -s $DRIVE_PATH $SYM_PATH




In [ ]:
#@title apt install requirements

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

from IPython.display import clear_output

!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg xvfb
!pip3 install pyvirtualdisplay
!pip install gymnasium


clear_output()

In [ ]:
#@title clone course repo

%cd $SYM_PATH
# !git clone {repo_url}
!git clone --single-branch --branch main https://github.com/CLAIR-LAB-TECHNION/CLAI.git
%cd CLAI/tutorials/dqn_tutorial

from fqi.collect_data import collect_data, load_data, save_data
from dqn.replay_buffer import ReplayBufferSamples
from dqn.evaluation import evaluate_policy
from notebook_utils import show_videos

clear_output()

In [ ]:
#@title set up virtual display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

## Import the packages

In addition to the installed libraries, we also use:

- `random`: To generate random numbers (that will be useful for epsilon-greedy policy).
- `imageio`: To generate a replay video.

In [ ]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

from tqdm.notebook import tqdm




# <img src="https://img.icons8.com/?size=50&id=42948&format=png&color=000000" style="height:30px;display:inline"> The CartPole-v1 Environment



The [CartPole-v1 environment](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py) is a classic control task in reinforcement learning. It is designed to test the basic capabilities of an RL algorithm in a simple yet challenging setting. Here’s a detailed description of the environment:

A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart, and the goal is to balance the pole by applying forces in the left and right direction on the cart.

The agent must learn to push the cart left or right **to keep the pole balanced and prevent it from falling over**.


<img src='https://gymnasium.farama.org/_images/cart_pole.gif' width=800/>

## <img src="https://img.icons8.com/?size=50&id=46589&format=png&color=000000" style="height:30px;display:inline"> Task 1: Getting Familiar with the CartPole Environment



<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/task_sign.png?raw=true' width=800/>

In this task, you will explore the CartPole-v1 environment. Your objective is to understand the basic operations of this environment, such as resetting the environment, taking actions, and observing the outcomes.

Questions:
- What are the dimensions of the state space?
- What are the possible actions in the action space?
- What triggers the termination of an episode in CartPole-v1?

In [ ]:
env_id = "CartPole-v1"
# Create the env
env = gym.make(env_id)


#### <img src="https://img.icons8.com/?size=50&id=42816&format=png&color=000000" style="height:30px;display:inline">  Solution



In [ ]:
# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

The state space of the CartPole-v1 environment is represented by a 4-dimensional vector:

1. **Cart Position**: The position of the cart along the track.
2. **Cart Velocity**: The velocity of the cart.
3. **Pole Angle**: The angle of the pole with respect to the vertical position.
4. **Pole Angular Velocity**: The rate of change of the pole's angle.

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

The action space is **discrete**, consisting of two possible actions:

1. **Push Left**: Apply a force to the left.
2. **Push Right**: Apply a force to the right.

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

The agent receives a reward 💰 of +1 for every timestep that the pole remains upright. The goal is to maximize the total reward over an episode.

An episode ends if any of the following conditions are met:

- The pole angle exceeds ±12 degrees.
- The cart position exceeds ±2.4 units from the center.
- The episode length exceeds 500 timesteps.

# Reminder About Tabular Q-Learning

Before diving into Fitted Q-Iteration (FQI), let's briefly revisit Tabular Q-learning, which we covered in the last tutorial. Tabular Q-learning is a fundamental reinforcement learning algorithm that aims to learn the optimal action-value function, $Q^*$, which gives the maximum expected reward for each state-action pair.


### Core Concepts:

- **Q-Table**: In Tabular Q-learning, we maintain a Q-table, a 2D array where each cell $(𝑠,𝑎)$ contains the Q-value for state $𝑠$ and action $𝑎$.

- **Bellman Equation**: The Q-values are updated iteratively based on the Bellman equation:
  
  $$
  Q(s,a) \leftarrow Q(s,a) + \alpha \left[ r + \gamma \max_{a'} Q(s',a') - Q(s,a) \right]
  $$

- **Policy Improvement**: The policy is derived from the Q-table by selecting the action with the highest Q-value in each state, known as the 𝜖-greedy policy to balance exploration and exploitation.


### Steps in Tabular Q-Learning:

1. **Initialize Q-Table**: Start with an initial Q-table with all zeros or random values.

2. **For each episode**:
   - **Initialize the starting state**.
   - **For each step within the episode**:
     - Choose an action 𝑎 using the 𝜖-greedy policy.
     - Take action 𝑎, observe the reward 𝑟 and the next state 𝑠′.
     - Update the Q-value 𝑄(𝑠,𝑎) using the Bellman equation.
     - Transition to the next state 𝑠′.
     
3. **Repeat until convergence**.


Tabular Q-learning is simple and effective for environments with a small state and action space. However, it becomes impractical when dealing with large or continuous state-action spaces, which leads us to Fitted Q-Iteration (FQI), an approach to handle such scenarios using function approximation.

;;;

# <img src="https://img.icons8.com/?size=50&id=aauVMeTJQzjy&format=png&color=000000" style="height:30px;display:inline"> Fitted Q Iteration (FQI)







Fitted Q Iteration (FQI) is an algorithm that extends q-learning to continuous state space using function estimators.

It iteratively approximates the q-values for a given dataset of transitions (state, action, reward, next_state) by iteratively solving a regression problem.

Compared to later algorithms like DQN, FQI uses the whole dataset at every iteration (working on the whole batch instead of using minibatches).


<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/fqi/tabular_limit_2.png" width="500"/>
</div>

###  Collect the Dataset






The first step in fitted Q-iteration is to collect a dataset consisting of tuples $(s_i, a_i, s_i', r_i)$
using some policy. The algorithm works for a wide range of different policies, though not all policy choices are equally good. The principles will apply to any policy, and it certainly doesn't have to be the latest policy.

This function collects an offline dataset of transitions that will be used to train a model using the FQI algorithm.

See docstring of the function for what is expected as input/output.

In [ ]:
from dataclasses import dataclass

import numpy as np
from gymnasium import spaces


@dataclass
class OfflineData:
    """
    A class to store transitions.
    """

    observations: np.ndarray  # same as "state" in the theory
    next_observations: np.ndarray
    actions: np.ndarray
    rewards: np.ndarray
    terminateds: np.ndarray

In [ ]:
def collect_data(env_id: str, n_steps: int = 50_000) -> OfflineData:
    """
    Collect transitions using a random agent (sample action randomly).

    :param env_id: The name of the environment.
    :param n_steps: Number of steps to perform in the env.
    :return: The collected transitions.
    """
    # Create the Gym env
    env = gym.make(env_id)

    assert isinstance(env.observation_space, spaces.Box)
    # Numpy arrays (buffers) to collect the data
    observations = np.zeros((n_steps, *env.observation_space.shape))
    next_observations = np.zeros((n_steps, *env.observation_space.shape))
    # Discrete actions
    actions = np.zeros((n_steps, 1))
    rewards = np.zeros((n_steps,))
    terminateds = np.zeros((n_steps,))

    # Variable to know if the episode is over (done = terminated or truncated)
    done = False

    # Start the first episode
    obs, _ = env.reset()

    for idx in range(n_steps):
        # Sample a random action
        action = env.action_space.sample()
        # Step in the environment
        next_obs, reward, terminated, truncated, info_ = env.step(action)
        # Store the transition
        # Note: we only record true termination (timeouts/truncations are artificial terminations)
        observations[idx, :] = obs
        next_observations[idx, :] = next_obs
        actions[idx, :] = action
        rewards[idx] = reward
        terminateds[idx] = terminated

        # Update current observation
        obs = next_obs
        # Check if the episode is over
        done = terminated or truncated
        # Don't forget to reset the env at the end of an episode
        if done:
            obs, _ = env.reset()


    ### END OF YOUR CODE

    return OfflineData(
        observations,
        next_observations,
        actions,
        rewards,
        terminateds,
    )

Let's try the collect data method:

In [ ]:
env_id = "CartPole-v1"
n_steps = 50_000
# Collect transitions for n_steps
data = collect_data(env_id=env_id, n_steps=n_steps)

In [ ]:
# Check the length of the collected data
assert len(data.observations) == n_steps
assert len(data.actions) == n_steps
# Check that there are multiple episodes in the data
assert not np.all(data.terminateds)
assert np.any(data.terminateds)
# Check the shape of the collected data
if env_id == "CartPole-v1":
    assert data.observations.shape == (n_steps, 4)
    assert data.next_observations.shape == (n_steps, 4)
assert data.actions.shape == (n_steps, 1)
assert data.rewards.shape == (n_steps,)

In [ ]:
from pathlib import Path

output_filename = Path("../data") / f"{env_id}_data"
# Create folder if it doesn't exist
output_filename.parent.mkdir(parents=True, exist_ok=True)

# Save collected data using numpy
save_data(data, output_filename)

In [ ]:
from functools import partial
from pathlib import Path
from typing import Optional

import gymnasium as gym
import numpy as np
from gymnasium import spaces
from sklearn import tree
from sklearn.base import RegressorMixin
from sklearn.exceptions import NotFittedError
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

### Choosing a Model

With FQI, you can use any regression model.

Here we are choosing a [k-nearest neighbors regressor](https://scikit-learn.org/stable/modules/neighbors.html#regression), but one could choose a linear model, a decision tree, a neural network, ...

In [ ]:
# First choose the regressor
model_class = partial(KNeighborsRegressor, n_neighbors=30)  # LinearRegression, GradientBoostingRegressor

### Loading offline dataset

The offline dataset contains the transitions collected using a random agent.

In [ ]:

env_id = "CartPole-v1"
output_filename = Path("../data") / f"{env_id}_data.npz"
render_mode = "rgb_array"

# Create test environment
env = gym.make(env_id, render_mode=render_mode)

# Load saved transitions
data = load_data(output_filename)

### Calculate Target Values

For every transition that you sampled, calculate a target value. This involves taking the reward from the transition plus the discounted maximum Q-value of the next state. The formula for the target value $y_i$ is:

$$
y_i = R(s_i, a_i) + \gamma \max_{a_i'} Q_{\phi}(s_i', a_i')
$$

Here, $Q_{\phi}$ is your previous Q-function estimator parameterized by $\phi$.


<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/fqi/q_value_fqi.png" width="300"/>
</div>

### First Iteration of FQI

For $n = 0$, the initial training set is defined as:

- $x = (s_t, a_t)$
- $y = r_t$

We fit a regression model $f_\theta(x) = y$ to obtain $ Q^{n=0}_\theta(s, a) $

In [ ]:
# First iteration:
# The target q-value is the reward obtained
targets = data.rewards.copy()
# Create input for current observations and actions
# Concatenate the observations and actions
# so we can predict qf(s_t, a_t)
current_obs_input = np.concatenate((data.observations, data.actions), axis=1)
# Fit the estimator for the current target
model = model_class().fit(current_obs_input, targets)

### Function to predict Q-Values
 For efficiency, we are not working with one transition at a time but on a batch of transitions/observations. The resulting `q_values` are therefore a matrix of shape `(batch_size, n_actions)` where `batch_size` is the size of the batch of observations (aka number of observations used at once) and `n_actions` is the number of available actions (`n_actions=2` for CartPole, as we can only go left or right)

Below is a diagram explaining how the predicted `q_values` will be stored.

Row-wise, we get the q-values estimates for all possible actions but for one single observation (state):

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/sklearn/q_value_batch.png" width="500"/>
    <br>
</div>

Column-wise, we get the q-values estimates for a given action but for a batch of observations (states):
<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/sklearn/q_value_batch_2.png" width="400"/>
</div>

In [ ]:
def get_q_values(
    model: RegressorMixin,
    obs: np.ndarray,
    n_actions: int,
) -> np.ndarray:
    """
    Retrieve the q-values for a set of observations(=states in the theory).
    qf(s_t, action) for all possible actions.

    :param model: Q-value estimator
    :param obs: A batch of observations
    :param n_actions: Number of discrete actions.
    :return: The predicted q-values for the given observations
        (batch_size, n_actions)
    """
    batch_size = len(obs)
    q_values = np.zeros((batch_size, n_actions))

    # TODO: for every possible actions a:
    # 1. Create the regression model input $(s, a)$ for the action a
    # and states s (here a batch of observations)
    # 2. Predict the q-values for the batch of states
    # 3. Update q-values array for the current action a

    # Predict q-value for each action
    for action_idx in range(n_actions):
        # Note: we should do one hot encoding if not using CartPole (n_actions > 2)
        # Create a vector of size (batch_size, 1) for the current action
        # This allows to do batch prediction for all the provided observations
        actions = action_idx * np.ones((batch_size, 1))
        # Concatenate the observations and the actions to obtain
        # the input to the q-value estimator
        # you can use `np.concatenate()`
        model_input = np.concatenate((obs, actions), axis=1)
        # Predict q-values for the given observation/action combination
        # shape: (batch_size, 1)
        predicted_q_values = model.predict(model_input)
        # Update the q-values array for the current action
        q_values[:, action_idx] = predicted_q_values

    return q_values

Let's test it with a subset of the collected data:

In [ ]:
n_observations = 2
n_actions = int(env.action_space.n)

q_values = get_q_values(model, data.observations[:n_observations], n_actions)

assert q_values.shape == (n_observations, n_actions)

### Function to evaluate a model

A greedy policy $\pi(s)$ can be defined using the q-value:

$\pi(s) = argmax_{a \in A} Q(s, a)$.

It is the policy that takes the action with the highest q-value for a given state.

In [ ]:
import os

from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder


def evaluate(
    model: RegressorMixin,
    env: gym.Env,
    n_eval_episodes: int = 10,
    video_name: Optional[str] = None,
) -> None:
    episode_returns, episode_reward = [], 0.0
    total_episodes = 0
    done = False

    # Setup video recorder
    video_recorder = None
    if video_name is not None and env.render_mode == "rgb_array":
        os.makedirs("../logs/videos/", exist_ok=True)

        video_recorder = VideoRecorder(
            env=env,
            base_path=f"../logs/videos/{video_name}",
        )

    obs, _ = env.reset()
    # Number of discrete actions
    n_actions = int(env.action_space.n)
    assert isinstance(env.action_space, spaces.Discrete), "FQI only support discrete actions"

    while total_episodes < n_eval_episodes:
        # Record video
        if video_recorder is not None:
            video_recorder.capture_frame()


        # Retrieve the q-values for the current observation
        # you need to re-use `get_q_values()`
        # Note: you need to add a batch dimension to the observation
        # you can use `obs[np.newaxis, ...]` for that: (obs_dim,) -> (batch_size=1, obs_dim)
        q_values = get_q_values(
            model,
            obs[np.newaxis, ...],
            n_actions,
        )
        # Select the action that maximizes the q-value for each state
        # Don't forget to remove the batch dimension, you can `.item()` for that
        best_action = int(np.argmax(q_values, axis=1).item())

        # Send the action to the env
        obs, reward, terminated, truncated, _ = env.step(best_action)


        episode_reward += float(reward)

        done = terminated or truncated
        if done:
            episode_returns.append(episode_reward)
            episode_reward = 0.0
            total_episodes += 1
            current_obs, _ = env.reset()

    if video_recorder is not None:
        print(f"Saving video to {video_recorder.path}")
        video_recorder.close()

    print(f"Total reward = {np.mean(episode_returns):.2f} +/- {np.std(episode_returns):.2f}")

In [ ]:
# Evaluate the first iteration
evaluate(model, env, n_eval_episodes=10)

### Step 3: Train a New Q-Function



Train a new Q-function by finding a new parameter vector $\phi$ that minimizes the difference between the Q-values and the corresponding target values  $y_i$. The Q-function takes as input the state $s$ and action $a$ and outputs a scalar value.


<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/task_sign.png?raw=true' width=800/>

### <img src="https://img.icons8.com/?size=50&id=46589&format=png&color=000000" style="height:30px;display:inline"> Task: The Fitted Q Iterations

1. Create the training set based on the previous iteration $ Q^{n-1}_\theta(s, a) $ and the transitions:
- input: $x = (s_t, a_t)$
- if $s_{t+1}$ is non-terminal: $y = r_t + \gamma \cdot \max_{a' \in A}(Q^{n-1}_\theta(s_{t+1}, a'))$
- if $s_{t+1}$ is terminal, do not bootstrap: $y = r_t$

2. Fit a model $f_\theta$ using a regression algorithm to obtain $ Q^{n}_\theta(s, a)$

\begin{aligned}
 f_\theta(x) = y
\end{aligned}

4. Repeat, $n = n + 1$

First, let's define some constants:

In [ ]:
# Max number of iterations
n_iterations = 20
# How often do we evaluate the learned model
eval_freq = 2
# How many episodes to evaluate every eval-freq
n_eval_episodes = 10
# discount factor
gamma = 0.99
# Number of discrete actions
n_actions = int(env.action_space.n)

Then do several iteration of the FQI algorithm

**HINT**: For that exercise, you should take a look at the first iteration of FQI.

**HINT**: The`data` variable (containing the offline dataset) is an instance of `OfflineData`:

```python
@dataclass
class OfflineData:
    """
    A class to store transitions.
    """

    observations: np.ndarray  # same as "state" in the theory
    next_observations: np.ndarray
    actions: np.ndarray
    rewards: np.ndarray
    terminateds: np.ndarray
```

In [ ]:
# Load saved transitions
data = load_data(output_filename)

print(type(data))
print(data.observations.shape)

 We are working here with a batch of data. Thanks to numpy, you can efficiently compute the max over a batch:

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/sklearn/q_values_max.png" width="500"/>
</div>

**HINT**: Using masking, you can elegantly handle the two cases for the regression target. The value of $y$ depends on whether the episode is terminated or not (in one case we bootstrap using the q-value, in the other case, we do not bootstrap and use the terminal reward instead).

Visually, masking is an element-wise operation:

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/sklearn/masking.png" width="500"/>
</div>



In [ ]:
%%script true
for iter_idx in range(n_iterations):
    ### YOUR CODE HERE
    # TODO:
    # 1. Compute the q values for the next states using
    # the previous regression model
    # 2. Keep only the next q values that correspond
    # to the greedy-policy
    # 3. Construct the regression target (TD(0) target)
    # 4. Fit a new regression model with this new target

    # First, retrieve the q-values for the next states (data.next_observations)
    # for all possible actions
    # you need to use `get_q_values()` method

    # Follow-greedy policy: use the action with the highest q-value
    # to compute the next q-values

    # The new target is the reward + what our agent expect to get
    # if it follows a greedy policy (follow action with the highest q-value)
    # Reminder: you should not bootstrap if terminated=True
    # (you can mask the next q values for that using `np.logical_not`)


    # Update the q-value estimate for the current (observations, actions) pair
    # (current_obs_input)
    # with the current target,
    # i.e., fit a regression model using the new target
    # Note: you can use `model_class()` to instantiate a new model
    model =


    ### END OF YOUR CODE

    if (iter_idx + 1) % eval_freq == 0:
        print(f"Iter {iter_idx + 1}")
        print(f"Score: {model.score(current_obs_input, targets):.2f}")
        evaluate(model, env, n_eval_episodes)

#### <img src="https://img.icons8.com/?size=50&id=42816&format=png&color=000000" style="height:30px;display:inline">  Solution



In [ ]:
for iter_idx in range(n_iterations):
    ### YOUR CODE HERE
    # TODO:
    # 1. Compute the q values for the next states using
    # the previous regression model
    # 2. Keep only the next q values that correspond
    # to the greedy-policy
    # 3. Construct the regression target (TD(0) target)
    # 4. Fit a new regression model with this new target

    # Construct TD(0) target
    # using current model and the next observations

    # First, retrieve the q-values for the next states
    # for all possible actions
    # you need to use `get_q_values()` method
    next_q_values = get_q_values(
        model,
        data.next_observations,
        n_actions=n_actions,
    )
    # Follow-greedy policy: use the action with the highest q-value
    # to compute the next q-values
    next_q_values = next_q_values.max(axis=1)
    # The new target is the reward + what our agent expect to get
    # if it follows a greedy policy (follow action with the highest q-value)
    # Reminder: you should not bootstrap if terminated=True
    # (you can mask the next q values for that using `np.logical_not`)
    should_bootstrap = np.logical_not(data.terminateds) # (1 - data.terminateds)
    targets = data.rewards + gamma * should_bootstrap * next_q_values
    # Update the q-value estimate with the current target,
    # i.e., fit a regression model using the new target
    model = model_class().fit(current_obs_input, targets)

    ### END OF YOUR CODE

    if (iter_idx + 1) % eval_freq == 0:
        print(f"Iter {iter_idx + 1}")
        print(f"Score: {model.score(current_obs_input, targets):.2f}")
        evaluate(model, env, n_eval_episodes)

### Record a video of the trained agent

In [ ]:
eval_env = gym.make(env_id, render_mode="rgb_array")
video_name = f"FQI_{env_id}"
n_eval_episodes = 3

evaluate(model, eval_env, n_eval_episodes, video_name=video_name)

In [ ]:
print(f"FQI agent on {env_id} after {n_iterations} iterations:")
show_videos("../logs/videos/", prefix=video_name)

# <img src="https://img.icons8.com/?size=50&id=66365&format=png&color=000000" style="height:30px;display:inline"> Deep Q-Network (DQN)







## Introduction

In this section, we'll introduce the transition from Fitted Q Iteration (FQI) to Deep Q-Networks (DQN), highlighting the limitations of FQI and how DQN addresses these challenges.

Fitted Q-Iteration (FQI) generally uses regression models to approximate the Q-value function. However, Deep Q-Learning (DQN) leverages neural networks for several compelling reasons:

1. **Scalability**: Neural networks can handle large and continuous state spaces, making them suitable for complex environments where traditional regression models and tabular methods are impractical.
2. **Generalization**: Neural networks can generalize from seen states to unseen states, allowing the agent to make informed decisions even in states it has not encountered before.
3. **Feature Extraction**: Especially in environments with raw sensory inputs (e.g., images), neural networks can learn useful features from raw data, improving the agent's decision-making process.


### Limitations of FQI

While FQI provides a solid foundation for value-based RL, it suffers from several limitations:

1. **Offline Reinforcement Learning**: FQI operates in an offline manner, meaning it uses a fixed dataset for training and does not continuously interact with the environment. This limits the ability of the algorithm to adapt to new data and learn incrementally.
2. **Loop Over All Actions**: To compute the target values, FQI needs to loop over all possible actions to find the maximum Q-value for the next state. This becomes computationally expensive in environments with large or continuous action spaces.
3. **Instability**: FQI can suffer from instability in learning because the target values used for training the Q-function are based on the same Q-function being learned. This can lead to oscillations and divergence in the Q-value estimates.

## <img src="https://img.icons8.com/?size=50&id=46457&format=png&color=000000" style="height:30px;display:inline"> Offline vs. Online Reinforcement Learning Trade-offs



Offline RL, also known as batch RL, involves learning a policy from a fixed dataset of experiences collected beforehand. The agent does not interact with the environment during training; instead, it learns solely from the provided dataset.

### Advantages:

- **Safety**: Offline RL is suitable for scenarios where interacting with the environment is costly, dangerous, or impractical (e.g., healthcare, autonomous driving).
- **Reproducibility**: Since the dataset is fixed, experiments are more reproducible, making it easier to benchmark different algorithms.

### Disadvantages:

- **Limited Exploration**: The agent can only learn from the experiences present in the dataset. If the dataset does not cover diverse scenarios, the learned policy may be suboptimal.
- **Overfitting**: There is a risk of overfitting to the fixed dataset, especially if the dataset is not representative of the entire state-action space.

Online RL, in contrast, involves continuous interaction with the environment. The agent collects new experiences while learning, allowing it to adapt and improve its policy over time.


### Online Q-Iteration

We can easily make our Fitted Q-Iteration (FQI) online by integrating it with real-time interaction with the environment. In online Q iterations, the agent continually updates its Q-values based on the latest interactions. The process can be described as follows:

<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/online_q_learing.png?raw=true' width=800/>

### Problems in the Online Version Without Replay Buffer and Target Network

Despite the potential of online Q-learning, it faces significant challenges when implemented without additional mechanisms such as a replay buffer and a target network:

1. **Correlated Samples**: Sequential experiences are often highly correlated, violating the assumption of independent and identically distributed (i.i.d.) samples necessary for effective stochastic gradient descent.
2. **Instability**: The targets used for updates are constantly changing because the Q-values depend on the current estimates. This can cause the learning process to become unstable and may prevent convergence.
3. **Overestimation Bias**: The Q-learning algorithm tends to overestimate Q-values due to the maximization step, leading to overly optimistic value estimates.

To address these issues, we need to introduce additional components that can help stabilize and improve the learning process. In the following sections, we will discuss these components in detail.


## <img src="https://img.icons8.com/?size=50&id=46958&format=png&color=000000" style="height:30px;display:inline"> DQN Components



## <img src="https://img.icons8.com/?size=50&id=46683&format=png&color=000000" style="height:30px;display:inline">  Replay Buffer



The replay buffer is one of the main component of DQN. It contains a collection of transitions, the same way we had a fixed dataset of transitions with FQI. However, compared to FQI, this ring buffer is consistently updated with new experience (and old transitions are dropped when the max capicity is reached).

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/dqn/replay_buffer.png" width="800"/>
</div>

To update the Q-Network, DQN samples mini-batches from the replay buffer (vs the whole dataset for FQI).

Each mini-batch can be represented using this structure:

```python
@dataclass
class ReplayBufferSamples:
    """
    A dataclass containing transitions from the replay buffer.
    """

    observations: np.ndarray  # same as states in the theory
    next_observations: np.ndarray
    actions: np.ndarray
    rewards: np.ndarray
    terminateds: np.ndarray
```

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/dqn/replay_buffer_sampling.png" width="600"/>
</div>


### Why Use a Replay Buffer?

1. **Breaking Correlations**: By storing experiences in a buffer and sampling them randomly, the replay buffer helps to break the temporal correlations between consecutive experiences. This decorrelation is vital for stabilizing the training of neural networks.

2. **Improving Data Efficiency**: The replay buffer allows the algorithm to reuse past experiences multiple times. This reusability is crucial in reinforcement learning where collecting new data can be expensive or time-consuming.

3. **Smoothing Training Data Distribution**: Sampling from a buffer ensures that the distribution of training data remains relatively stable over time, which helps in preventing the neural network from overfitting to recent experiences.

In [ ]:
from typing import Optional

import numpy as np
import torch as th
from gymnasium import spaces


### Trade-offs about the Size of the Replay Buffer

The size of the replay buffer presents a trade-off between memory usage and the diversity of experiences stored. A larger buffer can store a wider variety of experiences, improving the generalization capability of the neural network and reducing the likelihood of overfitting to specific patterns. However, it requires more memory and might retain older experiences that become less relevant as the agent learns and the environment dynamics change. On the other hand, a smaller buffer requires less memory and is more likely to sample recent experiences, which can be more relevant to the current policy. However, it captures a less diverse range of experiences, potentially limiting the neural network's ability to generalize and increasing the risk of sampling correlated experiences, which can destabilize training.

<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/task_sign.png?raw=true' width=800/>

### <img src="https://img.icons8.com/?size=50&id=46589&format=png&color=000000" style="height:30px;display:inline"> Task: Write the replay buffer



In [ ]:
class ReplayBuffer:
    """
    A simple replay buffer class to store and sample transitions.

    :param buffer_size: Max number of transitions to store
    :param observation_space: Observation space of the env,
        contains information about the observation type and shape.
    :param action_space: Action space of the env,
        contains information about the number of actions.
    """

    def __init__(
        self,
        buffer_size: int,
        observation_space: spaces.Box,
        action_space: spaces.Discrete,
    ) -> None:
        # Current position in the ring buffer
        self.current_idx = 0
        # Replay buffer max capacity
        self.buffer_size = buffer_size
        # Boolean flag to know when the buffer has reached its maximal capacity
        self.is_full = False

        self.observation_space = observation_space
        self.action_space = action_space
        # Create the different buffers
        self.observations = np.zeros((buffer_size, *observation_space.shape), dtype=observation_space.dtype)
        self.next_observations = np.zeros((buffer_size, *observation_space.shape), dtype=observation_space.dtype)
        # The action is an integer
        action_dim = 1
        self.actions = np.zeros((buffer_size, action_dim), dtype=action_space.dtype)

        ### YOUR CODE HERE

        # TODO: create the buffers (numpy arrays) for the rewards (dtype=np.float32)
        # and the terminated signals (dtype=bool)
        self.rewards = ...
        self.terminateds = ...

        ### END OF YOUR CODE

    def store_transition(
        self,
        obs: np.ndarray,
        next_obs: np.ndarray,
        action: int,
        reward: float,
        terminated: bool,
    ) -> None:
        """
        Store one transition in the buffer.

        :param obs: Current observation
        :param next_obs: Next observation
        :param action: Action taken for the current observation
        :param reward: Reward received after taking the action
        :param terminated: Whether it is the end of an episode or not
            (discarding episode truncation like timeout)
        """
        ### YOUR CODE HERE

        # TODO:
        # 1. Update the different buffers defined in the __init__
        # 2. Update the pointer (`self.current_idx`). Be careful,
        #  the pointer needs to be set to zero when reaching the end of the ring buffer.

        # Update the buffers to store the new transition


        # Update the pointer

        # If the buffer is full, we start from zero again, this is a ring buffer
        # you also need to set the flag `is_full` to True (so we know the buffer has reached its max capacity)


        ### END OF YOUR CODE

    def sample(self, batch_size: int) -> ReplayBufferSamples:
        """
        Sample with replacement `batch_size` transitions from the buffer.

        :param batch_size: How many transitions to sample.
        :return: Samples from the replay buffer
        """
        ### YOUR CODE HERE

        # TODO:
        # 1. Retrieve the upper bound (max index that can be sampled)
        #  it corresponds to `self.buffer_size` when the ring buffer is full (we can samples all indices)
        # 2. Sample `batch_size` indices with replacement from the buffer
        # (in the range [0, upper_bound[ ), numpy has a method `np.random.randint` for that ;)
        upper_bound = ...
        batch_indices = ...

        ### END OF YOUR CODE

        return ReplayBufferSamples(
            self.observations[batch_indices],
            self.next_observations[batch_indices],
            self.actions[batch_indices],
            self.rewards[batch_indices],
            self.terminateds[batch_indices],
        )

<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/solution_sign.png?raw=true' width=800/>

#### <img src="https://img.icons8.com/?size=50&id=42816&format=png&color=000000" style="height:30px;display:inline">  Solution


In [ ]:
class ReplayBuffer:
    """
    A simple replay buffer class to store and sample transitions.

    :param buffer_size: Max number of transitions to store
    :param observation_space: Observation space of the env,
        contains information about the observation type and shape.
    :param action_space: Action space of the env,
        contains information about the number of actions.
    """

    def __init__(
        self,
        buffer_size: int,
        observation_space: spaces.Box,
        action_space: spaces.Discrete,
    ) -> None:
        # Current position in the ring buffer
        self.current_idx = 0
        self.buffer_size = buffer_size
        # Boolean flag to know when the buffer has reached its maximal capacity
        self.is_full = False

        self.observation_space = observation_space
        self.action_space = action_space
        # Create the different buffers
        self.observations = np.zeros((buffer_size, *observation_space.shape), dtype=observation_space.dtype)
        self.next_observations = np.zeros((buffer_size, *observation_space.shape), dtype=observation_space.dtype)
        # The action is an integer
        action_dim = 1
        self.actions = np.zeros((buffer_size, action_dim), dtype=action_space.dtype)

        ### YOUR CODE HERE

        # TODO: create the buffers (numpy arrays) for the rewards (dtype=np.float32)
        # and the terminated signals (dtype=bool)
        self.rewards = np.zeros((buffer_size,), dtype=np.float32)
        self.terminateds = np.zeros((buffer_size,), dtype=bool)

        ### END OF YOUR CODE

    def store_transition(
        self,
        obs: np.ndarray,
        next_obs: np.ndarray,
        action: int,
        reward: float,
        terminated: bool,
    ) -> None:
        """
        Store one transition in the buffer.

        :param obs: Current observation
        :param next_obs: Next observation
        :param action: Action taken for the current observation
        :param reward: Reward received after taking the action
        :param terminated: Whether it is the end of an episode or not
            (discarding episode truncation like timeout)
        """
        ### YOUR CODE HERE

        # TODO:
        # 1. Update the different buffers defined in the __init__
        # 2. Update the pointer (`self.current_idx`), careful
        # the pointer need to be set to zero when reaching the end of the ring buffer

        # Update the buffers to store the new transition
        self.observations[self.current_idx] = obs
        self.next_observations[self.current_idx] = next_obs
        self.actions[self.current_idx] = action
        self.rewards[self.current_idx] = reward
        self.terminateds[self.current_idx] = terminated

        # Update the pointer
        self.current_idx += 1
        # If the buffer is full, we start from zero again, this is a ring buffer
        # you also need to set the flag `is_full` to True (so we know the buffer has reached its max capacity)
        if self.current_idx == self.buffer_size:
            self.is_full = True
            self.current_idx = 0

        ### END OF YOUR CODE

    def sample(self, batch_size: int) -> ReplayBufferSamples:
        """
        Sample with replacement `batch_size` transitions from the buffer.

        :param batch_size: How many transitions to sample.
        :return: Samples from the replay buffer
        """
        ### YOUR CODE HERE

        # TODO:
        # 1. Retrieve the upper bound (max index that can be sampled)
        #  it corresponds to `self.buffer_size` when the ring buffer is full (we can samples all indices)
        # 2. Sample `batch_size` indices with replacement from the buffer
        # (in the range [0, upper_bound[ ), numpy has a method `np.random.randint` for that ;)
        upper_bound = self.buffer_size if self.is_full else self.current_idx
        batch_indices = np.random.randint(0, upper_bound, size=batch_size)

        ### END OF YOUR CODE

        return ReplayBufferSamples(
            self.observations[batch_indices],
            self.next_observations[batch_indices],
            self.actions[batch_indices],
            self.rewards[batch_indices],
            self.terminateds[batch_indices],
        )

testing the replay buffer, without reaching max capacity:

In [ ]:
import gymnasium as gym

env = gym.make("CartPole-v1")

buffer_size = 1000
buffer = ReplayBuffer(buffer_size, env.observation_space, env.action_space)
obs, _ = env.reset()
# Fill the buffer
for _ in range(500):
    action = int(env.action_space.sample())
    next_obs, reward, terminated, truncated, _ = env.step(action)
    # Store new transition in the replay buffer
    buffer.store_transition(obs, next_obs, action, float(reward), terminated)
    # Update current observation
    obs = next_obs

    done = terminated or truncated
    if done:
        obs, _ = env.reset()

assert not buffer.is_full
assert buffer.current_idx == 500
samples = buffer.sample(batch_size=10)
assert len(samples.observations) == 10
assert samples.actions.shape == (10, 1)

Testing the replay buffer when reaching max capacity:

In [ ]:
# Fill the buffer completely
for _ in range(1000):
    action = int(env.action_space.sample())
    next_obs, reward, terminated, truncated, _ = env.step(action)
    buffer.store_transition(obs, next_obs, action, float(reward), terminated)
    # Update current observation
    obs = next_obs

    done = terminated or truncated
    if done:
        obs, _ = env.reset()

assert buffer.is_full
# We did a full loop
assert buffer.current_idx == 500
# Check sampling with replacement
# we should be able to sample more transitions
# than the capacity of the ring buffer
samples = buffer.sample(batch_size=1001)
assert len(samples.observations) == 1001
assert samples.actions.shape == (1001, 1)

## <img src="https://img.icons8.com/?size=50&id=110865&format=png&color=000000" style="height:30px;display:inline">  Q-Network

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/dqn/q_network.png" width="600"/>
</div>

Similiar to FQI, the Q-Network estimates $Q_\pi(s_t, a_t)$, however, instead of taking the action as input, it outputs q-values for all possible actions as output.

<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/task_sign.png?raw=true' width=800/>

### <img src="https://img.icons8.com/?size=50&id=46589&format=png&color=000000" style="height:30px;display:inline"> Task: Write the Q-Network using PyTorch

In [ ]:
from typing import Type

import torch as th
import torch.nn as nn
from gymnasium import spaces


class QNetwork(nn.Module):
    """
    A Q-Network for the DQN algorithm
    to estimate the q-value for a given observation.

    :param observation_space: Observation space of the env,
        contains information about the observation type and shape.
    :param action_space: Action space of the env,
        contains information about the number of actions.
    :param n_hidden_units: Number of units for each hidden layer.
    :param activation_fn: Activation function (ReLU by default)
    """

    def __init__(
        self,
        observation_space: spaces.Box,
        action_space: spaces.Discrete,
        n_hidden_units: int = 64,
        activation_fn: Type[nn.Module] = nn.ReLU,
    ) -> None:
        super().__init__()
        # Assume 1d space
        obs_dim = observation_space.shape[0]

        ### YOUR CODE HERE
        # TODO:
        # 1. Retrieve the number of discrete actions,
        # that will be the number of ouputs of the q-network
        # 2. Create the q-network, it will be a two layers fully-connected
        # neural network which take the state (observation) as input
        # and outputs the q-values for all possible actions

        # Retrieve the number of discrete actions (using attribute `n` from `action_space`)


        # Create the q network: a 2 fully connected hidden layers with `n_hidden_units` each
        # with `activation_fn` for the activation function after each hidden layer.
        # You should use `nn.Sequential` (combine several layers to create a network)
        # `nn.Linear` (fully connected layer) from PyTorch.
        self.q_net = ...

        ### END OF YOUR CODE

    def forward(self, observations: th.Tensor) -> th.Tensor:
        """
        :param observations: A batch of observation (batch_size, obs_dim)
        :return: The Q-values for the given observations
            for all the action (batch_size, n_actions)
        """
        return self.q_net(observations)

<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/solution_sign.png?raw=true' width=800/>

#### <img src="https://img.icons8.com/?size=50&id=42816&format=png&color=000000" style="height:30px;display:inline">  Solution


In [ ]:
from typing import Type

import torch as th
import torch.nn as nn
from gymnasium import spaces


class QNetwork(nn.Module):
    """
    A Q-Network for the DQN algorithm
    to estimate the q-value for a given observation.

    :param observation_space: Observation space of the env,
        contains information about the observation type and shape.
    :param action_space: Action space of the env,
        contains information about the number of actions.
    :param n_hidden_units: Number of units for each hidden layer.
    :param activation_fn: Activation function (ReLU by default)
    """

    def __init__(
        self,
        observation_space: spaces.Box,
        action_space: spaces.Discrete,
        n_hidden_units: int = 64,
        activation_fn: Type[nn.Module] = nn.ReLU,
    ) -> None:
        super().__init__()
        # Assume 1d space
        obs_dim = observation_space.shape[0]

        ### YOUR CODE HERE
        # TODO:
        # 1. Retrieve the number of discrete actions,
        # that will be the number of ouputs of the q-network
        # 2. Create the q-network, it will be a two layers fully-connected
        # neural network which take the state (observation) as input
        # and outputs the q-values for all possible actions

        # Retrieve the number of discrete actions (using attribute `n` from `action_space`)
        n_actions = int(action_space.n)

        # Create the q network: a 2 fully connected hidden layers with `n_hidden_units` each
        # with `activation_fn` for the activation function after each hidden layer.
        # You should use `nn.Sequential` (combine several layers to create a network)
        # `nn.Linear` (fully connected layer) from PyTorch.
        self.q_net = nn.Sequential(
            nn.Linear(obs_dim, n_hidden_units),
            activation_fn(),
            nn.Linear(n_hidden_units, n_hidden_units),
            activation_fn(),
            nn.Linear(n_hidden_units, n_actions),
        )

        ### END OF YOUR CODE

    def forward(self, observations: th.Tensor) -> th.Tensor:
        """
        :param observations: A batch of observation (batch_size, obs_dim)
        :return: The Q-values for the given observations
            for all the action (batch_size, n_actions)
        """
        return self.q_net(observations)

To test your Q-Network:

In [ ]:
env = gym.make("CartPole-v1")
q_net = QNetwork(env.observation_space, env.action_space)

print(q_net)
assert isinstance(q_net.q_net, nn.Sequential)
assert len(q_net.q_net) == 5
assert isinstance(q_net.q_net[0], nn.Linear)
assert isinstance(q_net.q_net[1], nn.ReLU)
assert isinstance(q_net.q_net[-1], nn.Linear)

obs, _ = env.reset()

with th.no_grad():
    # Create the input to the Q-Network
    obs_tensor = th.as_tensor(obs[np.newaxis, ...])
    q_values = q_net(obs_tensor)

    assert q_values.shape == (1, 2)

    best_action = q_values.argmax().item()

    assert isinstance(best_action, int)

## <img src="https://img.icons8.com/?size=50&id=42832&format=png&color=000000" style="height:30px;display:inline"> Epsilon-greedy data collection



Effective exploration strategies are critical for the agent to learn an optimal policy, especially in complex environments with large state and action spaces. By managing the exploration-exploitation trade-off appropriately, DQN can achieve robust learning and better performance.

To handle the exploration-exploitation trade-off in DQN, a common strategy is the use of epsilon-greedy action selection. This strategy involves the following steps:

1. **Epsilon-Greedy Policy**: The agent selects a random action with probability epsilon ($ϵ$) and chooses the action that maximizes the Q-value with probability (1−$ϵ$). This ensures a balance between exploration and exploitation.

2. **Epsilon ($ϵ$)**: The exploration rate, which starts high to encourage exploration and is gradually reduced over time to favor exploitation as the agent becomes more knowledgeable about the environment.

3. **Decay of Epsilon**: Over the course of training, epsilon is decayed according to a schedule, reducing the exploration rate. Initially, the agent explores more to gather diverse experiences, and as learning progresses, it exploits the learned policy more frequently. A common approach is to linearly decay epsilon from a high value (e.g., 1.0) to a low value (e.g., 0.1).

<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/task_sign.png?raw=true' width=800/>

### <img src="https://img.icons8.com/?size=50&id=46589&format=png&color=000000" style="height:30px;display:inline"> Task: Write the epsilon-greedy action selection strategy

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/dqn/epsilon_greedy.png" width="600"/>
</div>

In [ ]:
def epsilon_greedy_action_selection(
    q_net: QNetwork,
    observation: np.ndarray,
    exploration_rate: float,
    action_space: spaces.Discrete,
    device: str = "cpu",
) -> int:
    """
    Select an action according to an espilon-greedy policy:
    with a probability of epsilon (`exploration_rate`),
    sample a random action, otherwise follow the best known action
    according to the q-value.

    :param observation: A single observation.
    :param q_net: Q-network for estimating the q value
    :param exploration_rate: Current rate of exploration (in [0, 1], 0 means no exploration),
        probability to select a random action,
        this is "epsilon".
    :param action_space: Action space of the env,
        contains information about the number of actions.
    :param device: PyTorch device
    :return: An action selected according to the epsilon-greedy policy.
    """
    ### YOUR CODE HERE
    # TODO:
    # 1. Toss a biased coin (you can use `np.random.rand()`)
    # to decide if the agent should take a random action or not
    # (with probability p=`exploration_rate`)
    # 2. Either take a random action (sample the action space)
    # or follow the greedy policy (take the action with the highest q-value)

    if ...:
        # Random action
        action = ...
    else:
        # Greedy action
        # We do not need to compute the gradient, so we use `with th.no_grad():`
        with th.no_grad():
            # Convert the input to PyTorch tensor and add a batch dimension (obs_dim,) -> (1, obs_dim)
            # you can use `th.as_tensor` and numpy `[np.newaxis, ...]`

            # Compute q values for all actions

            # Greedy policy: select action with the highest q value
            # you can use PyTorch `.argmax()` for that
            action = ...

    ### END OF YOUR CODE

    return action

<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/solution_sign.png?raw=true' width=800/>

#### <img src="https://img.icons8.com/?size=50&id=42816&format=png&color=000000" style="height:30px;display:inline">  Solution

In [ ]:
def epsilon_greedy_action_selection(
    q_net: QNetwork,
    observation: np.ndarray,
    exploration_rate: float,
    action_space: spaces.Discrete,
    device: str = "cpu",
) -> int:
    """
    Select an action according to an espilon-greedy policy:
    with a probability of epsilon (`exploration_rate`),
    sample a random action, otherwise follow the best known action
    according to the q-value.

    :param observation: A single observation.
    :param q_net: Q-network for estimating the q value
    :param exploration_rate: Current rate of exploration (in [0, 1], 0 means no exploration),
        probability to select a random action,
        this is "epsilon".
    :param action_space: Action space of the env,
        contains information about the number of actions.
    :param device: PyTorch device
    :return: An action selected according to the epsilon-greedy policy.
    """
    ### YOUR CODE HERE
    # TODO:
    # 1. Toss a biased coin (you can use `np.random.rand()`)
    # to decide if the agent should take a random action or not
    # (with probability p=`exploration_rate`)
    # 2. Either take a random action (sample the action space)
    # or follow the greedy policy (take the action with the highest q-value)

    if np.random.rand() < exploration_rate:
        # Random action
        action = int(action_space.sample())
    else:
        # Greedy action
        # We do not need to compute the gradient, so we use `with th.no_grad():`
        with th.no_grad():
            # Convert the input to PyTorch tensor and add a batch dimension (obs_dim,) -> (1, obs_dim)
            # you can use `th.as_tensor` and numpy `[np.newaxis, ...]`
            obs_tensor = th.as_tensor(observation[np.newaxis, ...], device=device)
            # Compute q values for all actions
            q_values = q_net(obs_tensor)
            # Greedy policy: select action with the highest q value
            # you can use PyTorch `.argmax()` for that
            action = q_values.argmax().item()

    ### END OF YOUR CODE

    return action

In [ ]:
def collect_one_step(
    env: gym.Env,
    q_net: QNetwork,
    replay_buffer: ReplayBuffer,
    obs: np.ndarray,
    exploration_rate: float = 0.1,
    verbose: int = 0,
) -> np.ndarray:
    """
    Collect one transition and fill the replay buffer following an epsilon greedy policy.

    :param env: The environment object.
    :param q_net: Q-network for estimating the q value
    :param replay_buffer: Replay buffer to store the new transitions.
    :param obs: The current observation.
    :param exploration_rate: Current rate of exploration (in [0, 1], 0 means no exploration),
        probability to select a random action,
        this is "epsilon".
    :param verbose: The verbosity level (1 to print some info).
    :return: The last observation (important when collecting data multiple times).
    """
    ### YOUR CODE HERE

    # Select an action following an epsilon-greedy policy
    # you should use `epsilon_greedy_action_selection()`
    action = epsilon_greedy_action_selection(q_net, obs, exploration_rate, env.action_space)
    # Step in the env
    next_obs, reward, terminated, truncated, info = env.step(action)
    # Store the transition in the replay buffer
    replay_buffer.store_transition(obs, next_obs, action, float(reward), terminated)
    # Update current observation
    obs = next_obs

    ### END OF YOUR CODE

    if "episode" in info and verbose >= 1:
        print(f"Episode return={float(info['episode']['r']):.2f} length={int(info['episode']['l'])}")

    done = terminated or truncated
    if done:
        # Don't forget to reset the env at the end of an episode
        obs, _ = env.reset()
    return obs

Let's test the data collection:

In [ ]:
env = gym.make("CartPole-v1")
q_net = QNetwork(env.observation_space, env.action_space)
buffer = ReplayBuffer(2000, env.observation_space, env.action_space)

obs, _ = env.reset()
for _ in range(1000):
    obs = collect_one_step(env, q_net, buffer, obs, exploration_rate=0.1)

# Check current buffer position
assert buffer.current_idx == 1000

# Collect more data
for _ in range(1000):
    obs = collect_one_step(env, q_net, buffer, obs, exploration_rate=0.1)

# Buffer is full
assert buffer.current_idx == 0
assert buffer.is_full

### <img src="https://img.icons8.com/?size=50&id=49296&format=png&color=000000" style="height:30px;display:inline"> Exploration Schedule

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/dqn/linear_schedule.png" width="600"/>
</div>


In [ ]:
def linear_schedule(initial_value: float, final_value: float, current_step: int, max_steps: int) -> float:
    """
    Linear schedule for the exploration rate (epsilon).
    Note: we clip the value so the schedule is constant after reaching the final value
    at `max_steps`.

    :param initial_value: Initial value of the schedule.
    :param final_value: Final value of the schedule.
    :param current_step: Current step of the schedule.
    :param max_steps: Maximum number of steps of the schedule.
    :return: The current value of the schedule.
    """


    # Compute current progress (in [0, 1], 0 being the start)
    progress = current_step / max_steps
    # Clip the progress so the schedule is constant after reaching the final value
    progress = min(progress, 1.0)
    current_value = initial_value + progress * (final_value - initial_value)



    return current_value

To test the linear schedule:

In [ ]:
# Linear schedule
exploration_initial_eps = 1.0
exploration_final_eps = 0.01
exploration_rate = exploration_initial_eps
n_steps = 100
for step in range(n_steps + 1):
    exploration_rate = linear_schedule(exploration_initial_eps, exploration_final_eps, step, n_steps)
    if step == 0:
        assert exploration_rate == exploration_initial_eps

    obs = collect_one_step(env, q_net, buffer, obs, exploration_rate=exploration_rate)

assert np.allclose(exploration_rate, exploration_final_eps)

## DQN Update rule (no target network)
<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/dqn/annotated_dqn.png" width="1000"/>
</div>


## <img src="https://img.icons8.com/?size=50&id=YgRtmfcjZcf8&format=png&color=000000" style="height:30px;display:inline"> Target Network

In Deep Q-Learning, the Target Q Network is a crucial component designed to stabilize the training process. It addresses the instability issues that arise due to the moving target problem when updating Q-values.

A Target Q Network is a separate neural network used to generate target Q-values for the training updates. It is a copy of the primary Q Network, but its parameters are updated less frequently. This means that while the primary Q Network is updated at each training step, the Target Q Network remains fixed for a certain number of steps, providing a stable target for the Q-value updates.

<div>
    <img src="https://araffin.github.io/slides/dqn-tutorial/images/dqn/target_q_network.png" width="1000"/>
</div>





The only things that is changing is when predicting the next q value.

In DQN without target, the online network with weights **$\theta$** is used:

$y = r_t + \gamma \cdot \max_{a \in A}(\hat{Q}_{\pi}(s_{t+1}, a; \theta))$


whereas with DQN with target network, the target q-network (a delayed copy of the q-network) with weights **$\theta^\prime$** is used instead:

$y = r_t + \gamma \cdot \max_{a \in A}(\hat{Q}_{\pi}(s_{t+1}, a; \theta^\prime))$


## Why Use a Target Q Network?

1. **Stabilizing Training**: One of the main reasons to use a Target Q Network is to stabilize training. In Q-learning, the target for the Q-value update is derived from the same Q Network being updated, leading to a moving target problem. This can cause instability and divergence during training. By using a separate Target Q Network, the target values remain more stable over short periods, allowing for more reliable updates.

2. **Reducing Oscillations**: Without a Target Q Network, the learning process can oscillate or even diverge due to the constantly changing Q-values. The Target Q Network helps in smoothing these changes, providing a more consistent learning signal.

3. **Improving Convergence**: By decoupling the target Q-value generation from the learning Q-values, the agent can converge to a more optimal policy. The periodic updates to the Target Q Network ensure that the target values gradually follow the improvements in the Q Network, promoting stable learning.


<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/task_sign.png?raw=true' width=800/>

### <img src="https://img.icons8.com/?size=50&id=46589&format=png&color=000000" style="height:30px;display:inline">Task: Write the DQN update with target network



In [ ]:
%%script ture
def dqn_update(
    q_net: QNetwork,
    q_target_net: QNetwork,
    optimizer: th.optim.Optimizer,
    replay_buffer: ReplayBuffer,
    batch_size: int,
    gamma: float,
) -> None:
    """
    Perform one gradient step on the Q-network
    using the data from the replay buffer.

    :param q_net: The Q-network to update
    :param q_target_net: The target Q-network, to compute the td-target.
    :param optimizer: The optimizer to use
    :param replay_buffer: The replay buffer containing the transitions
    :param batch_size: The minibatch size, how many transitions to sample
    :param gamma: The discount factor
    """

    # Sample the replay buffer and convert them to PyTorch tensors
    replay_data = replay_buffer.sample(batch_size).to_torch()

    with th.no_grad():
        ### YOUR CODE HERE
        # TODO: use the target q-network instead of the online q-network
        # to compute the next values

        # Compute the Q-values for the next observations (batch_size, n_actions)
        # using the target network

        # Follow greedy policy: use the one with the highest value
        # (batch_size,)

        # If the episode is terminated, set the target to the reward

        # 1-step TD target

        ### END OF YOUR CODE

    # Get current Q-values estimates for the replay_data (batch_size, n_actions)
    q_values = q_net(replay_data.observations)
    # Select the Q-values corresponding to the actions that were selected
    # during data collection
    current_q_values = th.gather(q_values, dim=1, index=replay_data.actions)
    # Reshape from (batch_size, 1) to (batch_size,) to avoid broadcast error
    current_q_values = current_q_values.squeeze(dim=1)

    # Check for any shape/broadcast error
    # Current q-values must have the same shape as the TD target
    assert current_q_values.shape == (batch_size,), f"{current_q_values.shape} != {(batch_size,)}"
    assert current_q_values.shape == td_target.shape, f"{current_q_values.shape} != {td_target.shape}"

    # Compute the Mean Squared Error (MSE) loss
    # Optionally, one can use a Huber loss instead of the MSE loss
    loss = ((current_q_values - td_target) ** 2).mean()
    # Huber loss
    # loss = th.nn.functional.smooth_l1_loss(current_q_values, td_target)

    # Reset gradients
    optimizer.zero_grad()
    # Compute the gradients
    loss.backward()
    # Update the parameters of the q-network
    optimizer.step()

<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/solution_sign.png?raw=true' width=800/>

#### <img src="https://img.icons8.com/?size=50&id=42816&format=png&color=000000" style="height:30px;display:inline">  Solution

In [ ]:
def dqn_update(
    q_net: QNetwork,
    q_target_net: QNetwork,
    optimizer: th.optim.Optimizer,
    replay_buffer: ReplayBuffer,
    batch_size: int,
    gamma: float,
) -> None:
    """
    Perform one gradient step on the Q-network
    using the data from the replay buffer.

    :param q_net: The Q-network to update
    :param q_target_net: The target Q-network, to compute the td-target.
    :param optimizer: The optimizer to use
    :param replay_buffer: The replay buffer containing the transitions
    :param batch_size: The minibatch size, how many transitions to sample
    :param gamma: The discount factor
    """

    # Sample the replay buffer and convert them to PyTorch tensors
    replay_data = replay_buffer.sample(batch_size).to_torch()

    with th.no_grad():
        ### YOUR CODE HERE
        # TODO: use the target q-network instead of the online q-network
        # to compute the next values

        # Compute the Q-values for the next observations (batch_size, n_actions)
        # using the target network
        next_q_values = q_target_net(replay_data.next_observations)
        # Follow greedy policy: use the one with the highest value
        # (batch_size,)
        next_q_values, _ = next_q_values.max(dim=1)
        # If the episode is terminated, set the target to the reward
        should_bootstrap = th.logical_not(replay_data.terminateds)
        # 1-step TD target
        td_target = replay_data.rewards + gamma * next_q_values * should_bootstrap

        ### END OF YOUR CODE

    # Get current Q-values estimates for the replay_data (batch_size, n_actions)
    q_values = q_net(replay_data.observations)
    # Select the Q-values corresponding to the actions that were selected
    # during data collection
    current_q_values = th.gather(q_values, dim=1, index=replay_data.actions)
    # Reshape from (batch_size, 1) to (batch_size,) to avoid broadcast error
    current_q_values = current_q_values.squeeze(dim=1)

    # Check for any shape/broadcast error
    # Current q-values must have the same shape as the TD target
    assert current_q_values.shape == (batch_size,), f"{current_q_values.shape} != {(batch_size,)}"
    assert current_q_values.shape == td_target.shape, f"{current_q_values.shape} != {td_target.shape}"

    # Compute the Mean Squared Error (MSE) loss
    # Optionally, one can use a Huber loss instead of the MSE loss
    loss = ((current_q_values - td_target) ** 2).mean()
    # Huber loss
    # loss = th.nn.functional.smooth_l1_loss(current_q_values, td_target)

    # Reset gradients
    optimizer.zero_grad()
    # Compute the gradients
    loss.backward()
    # Update the parameters of the q-network
    optimizer.step()

## <img src="https://img.icons8.com/?size=50&id=46621&format=png&color=000000" style="height:30px;display:inline"> Updated training loop



<img src='https://github.com/CLAIR-LAB-TECHNION/CLAI/blob/main/tutorials/assets/fixed-q-target-pseudocode.jpg?raw=true' width=800/>

In [ ]:

def run_dqn(
    env_id: str = "CartPole-v1",
    replay_buffer_size: int = 50_000,
    # How often do we copy the parameters from the Q-network to the target network
    target_network_update_interval: int = 1000,
    # Warmup phase
    learning_starts: int = 100,
    # Exploration schedule
    # (for the epsilon-greedy data collection)
    exploration_initial_eps: float = 1.0,
    exploration_final_eps: float = 0.01,
    exploration_fraction: float = 0.1,
    n_timesteps: int = 20_000,
    update_interval: int = 2,
    learning_rate: float = 3e-4,
    batch_size: int = 64,
    gamma: float = 0.99,
    n_hidden_units: int = 64,
    n_eval_episodes: int = 10,
    evaluation_interval: int = 1000,
    eval_exploration_rate: float = 0.0,
    seed: int = 2023,
    # device: Union[th.device, str] = "cpu",
    eval_render_mode: Optional[str] = None,  # "human", "rgb_array", None
) -> QNetwork:
    """
    Run Deep Q-Learning (DQN) on a given environment.
    (with a target network)

    :param env_id: Name of the environment
    :param replay_buffer_size: Max capacity of the replay buffer
    :param target_network_update_interval: How often do we copy the parameters
         to the target network
    :param learning_starts: Warmup phase to fill the replay buffer
        before starting the optimization.
    :param exploration_initial_eps: The initial exploration rate
    :param exploration_final_eps: The final exploration rate
    :param exploration_fraction: The fraction of the number of steps
        during which the exploration rate is annealed from
        initial_eps to final_eps.
        After this many steps, the exploration rate remains constant.
    :param n_timesteps: Number of timesteps in total
    :param update_interval: How often to update the Q-network
        (every update_interval steps)
    :param learning_rate: The learning rate to use for the optimizer
    :param batch_size: The minibatch size
    :param gamma: The discount factor
    :param n_hidden_units: Number of units for each hidden layer
        of the Q-Network.
    :param n_eval_episodes: The number of episodes to evaluate the policy on
    :param evaluation_interval: How often to evaluate the policy
    :param eval_exploration_rate: The exploration rate to use during evaluation
    :param seed: Random seed for the pseudo random generator
    :param eval_render_mode: The render mode to use for evaluation
    """
    # Set seed for reproducibility
    # Seed Numpy as PyTorch pseudo random generators
    # Seed Numpy RNG
    np.random.seed(seed)
    # seed the RNG for all devices (both CPU and CUDA)
    th.manual_seed(seed)

    # Create the environment
    env = gym.make(env_id)
    # For highway env
    env = gym.wrappers.FlattenObservation(env)
    env = gym.wrappers.RecordEpisodeStatistics(env)
    assert isinstance(env.observation_space, spaces.Box)
    assert isinstance(env.action_space, spaces.Discrete)
    env.action_space.seed(seed)

    # Create the evaluation environment
    eval_env = gym.make(env_id, render_mode=eval_render_mode)
    eval_env = gym.wrappers.FlattenObservation(eval_env)
    eval_env.reset(seed=seed)
    eval_env.action_space.seed(seed)

    # Create the q-network
    q_net = QNetwork(env.observation_space, env.action_space, n_hidden_units=n_hidden_units)
    # Create the target network
    q_target_net = QNetwork(env.observation_space, env.action_space, n_hidden_units=n_hidden_units)
    # Copy the parameters of the q-network to the target network
    q_target_net.load_state_dict(q_net.state_dict())

    # For flappy bird
    if env.observation_space.dtype == np.float64:
        q_net.double()
        q_target_net.double()

    # Create the optimizer, we only optimize the parameters of the q-network
    optimizer = th.optim.Adam(q_net.parameters(), lr=learning_rate)

    # Create the Replay buffer
    replay_buffer = ReplayBuffer(replay_buffer_size, env.observation_space, env.action_space)
    # Reset the env
    obs, _ = env.reset(seed=seed)
    for current_step in range(1, n_timesteps + 1):
        # Update the current exploration schedule (update the value of epsilon)
        exploration_rate = linear_schedule(
            exploration_initial_eps,
            exploration_final_eps,
            current_step,
            int(exploration_fraction * n_timesteps),
        )
        # Do one step in the environment following an epsilon-greedy policy
        # and store the transition in the replay buffer
        obs = collect_one_step(
            env,
            q_net,
            replay_buffer,
            obs,
            exploration_rate=exploration_rate,
            verbose=0,
        )

        # Update the target network
        # by copying the parameters from the Q-network every target_network_update_interval steps
        if (current_step % target_network_update_interval) == 0:
            q_target_net.load_state_dict(q_net.state_dict())

        # Update the Q-network every update_interval steps
        # after learning_starts steps have passed (warmup phase)
        if (current_step % update_interval) == 0 and current_step > learning_starts:
            # Do one gradient step
            dqn_update(q_net, q_target_net, optimizer, replay_buffer, batch_size, gamma=gamma)

        if (current_step % evaluation_interval) == 0:
            print()
            print(f"Evaluation at step {current_step}:")
            print(f"exploration_rate={exploration_rate:.2f}")
            # Evaluate the current greedy policy (deterministic policy)
            evaluate_policy(eval_env, q_net, n_eval_episodes, eval_exploration_rate=eval_exploration_rate)
            # Save a checkpoint
            th.save(q_net.state_dict(), f"../logs/q_net_checkpoint_{env_id}_{current_step}.pth")
    return q_net

### Train DQN agent with target network on CartPole env

In [ ]:
# Tuned hyperparameters from the RL Zoo3 of the Stable Baselines3 library
# https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/dqn.yml

env_id = "CartPole-v1"

q_net = run_dqn(
    env_id=env_id,
    replay_buffer_size=100_000,
    # Note: you can remove the target network
    # by setting target_network_update_interval=1
    target_network_update_interval=10,
    learning_starts=1000,
    exploration_initial_eps=1.0,
    exploration_final_eps=0.04,
    exploration_fraction=0.1,
    n_timesteps=80_000,
    update_interval=2,
    learning_rate=1e-3,
    batch_size=64,
    gamma=0.99,
    n_eval_episodes=10,
    evaluation_interval=5000,
    # No exploration during evaluation
    # (deteministic policy)
    eval_exploration_rate=0.0,
    seed=2022,
)

## <img src="https://img.icons8.com/?size=50&id=46759&format=png&color=000000" style="height:30px;display:inline"> Visualize the trained agent



In [ ]:
eval_env = gym.make(env_id, render_mode="rgb_array")
n_eval_episodes = 3
eval_exploration_rate = 0.0
video_name = f"DQN_{env_id}"

# Optional: load checkpoint
# q_net = QNetwork(eval_env.observation_space, eval_env.action_space, n_hidden_units=64)
# q_net.load_state_dict(th.load("../logs/q_net_checkpoint_CartPole-v1_75000.pth"))

evaluate_policy(
    eval_env,
    q_net,
    n_eval_episodes,
    eval_exploration_rate=eval_exploration_rate,
    video_name=video_name,
)

show_videos("../logs/videos/", prefix=video_name)

## <img src="https://img.icons8.com/?size=50&id=y_hu-u4x2vis&format=png&color=000000" style="height:30px;display:inline"> Training DQN agent on flappy bird:

You can go in the [GitHub repo](https://github.com/araffin/flappy-bird-gymnasium/tree/patch-1) to learn more about this environment.

<div>
    <img src="https://raw.githubusercontent.com/markub3327/flappy-bird-gymnasium/main/imgs/dqn.gif" width="300"/>
</div>


In [ ]:
!pip install "flappy-bird-gymnasium @ git+https://github.com/araffin/flappy-bird-gymnasium@patch-1"

In [ ]:
import flappy_bird_gymnasium

In [ ]:
env_id = "FlappyBird-v0"

q_net = run_dqn(
    env_id=env_id,
    replay_buffer_size=100_000,
    # Note: you can remove the target network
    # by setting target_network_update_interval=1
    target_network_update_interval=250,
    learning_starts=10_000,
    exploration_initial_eps=1.0,
    exploration_final_eps=0.03,
    exploration_fraction=0.1,
    n_timesteps=500_000,
    update_interval=4,
    learning_rate=1e-3,
    batch_size=128,
    gamma=0.98,
    n_eval_episodes=5,
    evaluation_interval=50000,
    n_hidden_units=256,
    # No exploration during evaluation
    # (deteministic policy)
    eval_exploration_rate=0.0,
    seed=2023,
    eval_render_mode=None,
)

### Record a video of the trained agent

In [ ]:
eval_env = gym.make(env_id, render_mode="rgb_array")
n_eval_episodes = 3
eval_exploration_rate = 0.00
video_name = f"DQN_{env_id}"


# Optional: load checkpoint
q_net = QNetwork(eval_env.observation_space, eval_env.action_space, n_hidden_units=256)
# Convert weights from float32 to float64 to match flappy bird obs
q_net.double()
q_net.load_state_dict(th.load("../logs/q_net_checkpoint_FlappyBird-v0_200000.pth"))

evaluate_policy(
    eval_env,
    q_net,
    n_eval_episodes,
    eval_exploration_rate=eval_exploration_rate,
    video_name=video_name,
)

show_videos("../logs/videos/", prefix=video_name)

## <img src="https://img.icons8.com/?size=50&id=55162&format=png&color=000000" style="height:30px;display:inline"> Improving Q-Learning



### Overestimation in Q-Learning
One significant issue in Q-learning is overestimation. When calculating target values, Q-learning uses the maximum Q-value of possible actions, which can lead to overestimation. This occurs because the expected value of the maximum of multiple estimates is generally greater than the maximum of their expected values. Essentially, the noise in the Q-value estimates can lead to selecting actions that appear better than they are due to the variability in the estimates.

### Addressing Overestimation with Double Q-Learning
Double Q-learning addresses the overestimation problem by using two separate networks: one to select the action and the other to evaluate the action's value. This method helps decorrelate the noise in the action selection from the noise in the value evaluation. By doing so, it mitigates the overestimation issue, leading to more accurate Q-value predictions and more stable learning.

### Multi-Step Returns
Another improvement in Q-learning is the use of multi-step returns. Instead of relying solely on one-step backups, which are highly biased but have low variance, multi-step returns sum rewards over multiple steps. This approach reduces bias as the target values include more true rewards, thus providing a better learning signal. However, it increases variance since the summation over multiple steps introduces more noise.

### Off-Policy Considerations with Multi-Step Returns

Multi-step returns are only valid with on-policy data, where the policy used to generate the data matches the current policy being learned. With off-policy data, using multi-step returns can be problematic because the actions in the sampled data might not align with the new policy's actions. Several solutions exist for this issue:

1. **Ignoring the Problem**: This often works well in practice despite the theoretical drawbacks.
2. **Dynamically Cutting the Trace**: Adjust the number of steps, 𝑛, to match on-policy data, ensuring that the sampled actions align with what the policy would have done.
3. **Importance Sampling**: Use a stochastic policy and importance weighting to adjust the multi-step returns.


## <img src="https://img.icons8.com/?size=50&id=Vm9v3PaQFGMr&format=png&color=000000" style="height:30px;display:inline"> Q-Learning with Continuous Actions



So far, when we talked about Q-learning algorithms, we mainly focused on algorithms with discrete action spaces. It is actually possible but somewhat more complicated to extend Q-learning procedures to the case when we have continuous actions.

### The Problem with Continuous Actions

The problem is that when you select your actions, you need to perform this argmax. An argmax over discrete actions is pretty straightforward; you simply evaluate the Q-value for every possible action and take the best one. But when you have continuous actions, this is, of course, much harder. This comes up in two places: when evaluating the argmax policy and when computing the target value, which requires the max or, in the case of double Q-learning, also an argmax. The target value max is particularly problematic because that happens in the inner loop of training, so you really want it to be very fast and very efficient.

#### Continuous Optimization Procedure

A simple solution is to approximate the max over a continuous action as the max over a discrete set of actions that are sampled randomly. For instance, you could sample a set of $n$ actions, maybe uniformly at random from the set of valid actions, and then take the Q-value with the largest of those actions. This method is straightforward and efficient to parallelize but might suffer from inaccuracy as the action space dimensionality increases.

#### Using a Function Class with Easy Optimization

Another option is to use a function class that is inherently easy to optimize. For example, quadratic functions have closed-form solutions for their optima. The [NAF](https://towardsdatascience.com/applied-reinforcement-learning-v-normalized-advantage-function-naf-for-continuous-control-62ad143d3095) (Normalized Advantage Function) architecture is one such approach, where a neural network outputs a quadratic function in the action. This method simplifies the maximization operation but reduces the representational capacity of the Q-function.

#### Learning an Approximate Maximizer

The third option is to perform Q-learning with continuous actions by learning an approximate maximizer. This approach involves training a second neural network to perform the maximization. This method is used in algorithms like [DDPG](https://spinningup.openai.com/en/latest/algorithms/ddpg.html) (Deep Deterministic Policy Gradient), where a policy network approximates the argmax of the Q-function.

# <img src="https://img.icons8.com/?size=100&id=46509&format=png&color=000000" style="height:50px;display:inline"> Conclusion
---

In this tutorial, we covered the transition from Fitted Q-Iteration (FQI) to Deep Q-Networks (DQN). We started by understanding FQI as a regression problem designed to learn Q-values for state-action pairs. Despite its strengths, FQI faces limitations such as challenges in scaling, looping over all possible actions, and instability in target updates.

To address these issues, we moved to DQN, which uses neural networks to handle large state spaces, replay buffers to mitigate sample correlation, and target networks to stabilize learning. The trade-offs between offline and online reinforcement learning were also discussed, highlighting the advantages of an online approach for continuous learning.

The core components of DQN—Q-network, replay buffer, exploration vs. exploitation strategies, and target Q-network—each play a crucial role in overcoming the limitations of FQI, making DQN a powerful tool for complex reinforcement learning tasks. This tutorial has provided a concise understanding of these components and their importance in creating a stable and efficient reinforcement learning algorithm.






# <img src="https://img.icons8.com/dusk/64/000000/plus-2-math.png" style="height:50px;display:inline"> Further Reading
---


For those interested in diving deeper into Q-learning and its developments, here are some seminal and influential papers in the field:

### Classic Papers

- **Watkins (1989)**. ["Learning from Delayed Rewards"](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf): This paper introduces the Q-learning algorithm, a foundational work in reinforcement learning.
- **Riedmiller (2005)**. ["Neural Fitted Q-Iteration"](https://www.ias.informatik.tu-darmstadt.de/uploads/Team/JanPeters/riedmiller05nips.pdf): This paper discusses batch-mode Q-learning with neural networks, a significant step towards modern deep reinforcement learning.

### Deep Reinforcement Learning Q-Learning Papers

- **Lange & Riedmiller (2010)**. ["Deep Auto-Encoder Neural Networks in Reinforcement Learning"](https://ml.informatik.uni-freiburg.de/former/_media/publications/lange-2010-icann.pdf): An early image-based Q-learning method using autoencoders to construct embeddings.
- **Mnih et al. (2013)**. ["Human-Level Control through Deep Reinforcement Learning"](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf): This landmark paper introduces Q-learning with convolutional networks for playing Atari games, marking a significant advancement in deep reinforcement learning.
- **Van Hasselt, Guez, & Silver (2015)**. ["Deep Reinforcement Learning with Double Q-Learning"](https://arxiv.org/abs/1509.06461): This paper presents a very effective trick to improve the performance of deep Q-learning by addressing overestimation issues.
- **Lillicrap et al. (2016)**. ["Continuous Control with Deep Reinforcement Learning"](https://arxiv.org/abs/1509.02971): Discusses continuous Q-learning with an actor network for approximate maximization, known as DDPG.
- **Gu, Lillicrap, & Stuskever (2016)**. ["Continuous Deep Q-Learning with Model-Based Acceleration"](https://arxiv.org/abs/1603.00748): This paper explores continuous Q-learning with action-quadratic value functions, enhancing the efficiency of learning in continuous action spaces.
- **Wang et al. (2016)**. ["Dueling Network Architectures for Deep Reinforcement Learning"](https://arxiv.org/abs/1511.06581): Introduces an architecture that separates value and advantage estimation in the Q-function, improving stability and performance.






# <img src="https://img.icons8.com/?size=100&id=46756&format=png&color=000000" style="height:50px;display:inline"> Credits
---
* Examples and code snippets were taken from <a href="https://github.com/araffin/rlss23-dqn-tutorial/tree/main"> Reinforcement Learning Summer School 2023 - DQN Tutorial </a>
* Examples and explanations were taken from <a href="https://rail.eecs.berkeley.edu/deeprlcourse/">CS285 - Deep Reinforcement Learning Course at UC Berkeley</a>
* Icons from <a href="https://icons8.com/">Icons8.com